In [ ]:
import numpy as np
import pandas as pd
from NeuralNetwork import loss, activation, layer, network
import plotly.graph_objects as go
import plotly.express as px

# XOR Gate Prediction

In [ ]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

Y = np.array([[0], [1],
              [1], [0]])

In [ ]:
model = network.Sequential()
model.add_layer(layer.Dense(n_neurons=3, name='Layer_1', freeze_weights=False, weights=np.array([[0.1, 0.2, 0.3],
                                                                                                     [0.6, 0.4, 0.7]]),
                                bias=np.array([[0, 0, 0]])))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=1, name='Layer_2', freeze_weights=False, weights=np.array([[0.1],
                                                                                                     [0.4],
                                                                                                     [0.9]]),
                                bias = np.array([[0]])))

model.add_layer(activation.Sigmoid(name='Activation_2'))

print(model)
model.compile(loss=loss.MSE(), inputs=X, target=Y, batch=4)
print(model)

model.train(epochs=5000)

pred_val = model.predict(inputs=X)
print(pred_val)

# Quantile Regression Prediction Interval
Prediction Interval at 0.977 - 0.023 ~ 0.95

In [ ]:
def f_predictable(x):
    return x+np.sin(np.pi*x/2)


def f(x, std=0.2):
    return f_predictable(x)+np.random.randn(len(x))*std


def get_data(num, start=0, end=4):
        x = np.sort(np.random.rand(num)*(end-start)+start)
        y = f(x)
        return x.reshape(-1, 1), y

x_train, y_train = get_data(num=20000)
y_train = y_train.reshape(-1,1)
x_test, y_test = get_data(num=1000)
y_test = y_test.reshape(-1,1)

In [ ]:
model_upper = network.Sequential()

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_1', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_1'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_2', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_2'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_3', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_3'))

model_upper.add_layer(layer.Dense(n_neurons=1, name='Layer_4', freeze_weights=False, lr=0.01))

model_upper.compile(loss=loss.Quantile(quantile=0.977), inputs=x_train, target=y_train, batch=24)

model_upper.train(epochs=400)

In [ ]:
model_lower = network.Sequential()

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_1', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_1'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_2', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_2'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_3', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_3'))

model_lower.add_layer(layer.Dense(n_neurons=1, name='Layer_4', freeze_weights=False, lr=0.01))

model_lower.compile(loss=loss.Quantile(quantile=0.023), inputs=x_train, target=y_train, batch=24)

model_lower.train(epochs=400)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_train.reshape(-1), y=y_train.reshape(-1),
                    mode='markers',
                    name='Original Data'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_upper.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Upper Bound'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_lower.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Lower Bound'))


fig.show()

# Image Classification
Multi Class Prediction

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [2]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800)
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -1*np.ones((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

# Image Classification
Multi Label Prediction

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [1]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800)
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0.5, 1, 0)
np.where((pred_val == one_hot_labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -np.ones((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0, 1, -1)
np.where((pred_val == one_hot_labels)==False)

In [35]:
from numba import njit
from numba.experimental import jitclass
import numpy as np

In [3]:
class MSE():
    """MSE Loss Function
    """

    def forward(self, target, pred_val):
        """Forward propagation
        https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html

        Args:
            target (samples, n_features): Target value
            pred_val (samples, n_features): Predicted value
        """
        self.target = target
        self.pred_val = pred_val
        diff = self.target - self.pred_val
        self.output = np.sum(diff**2) / (self.target.size * 2)

    def backward(self):
        """Backward propagation
        https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html

        Returns:
            samples, n_features: Gradient of loss
        """
        diff = self.target - self.pred_val
        self.delta_grad = -1 * diff / self.target.size
        return self.delta_grad

In [4]:
testdata = np.random.randint(low=1,high=20, size=(30000,30000))
testdata

array([[ 6,  1,  4, ...,  9, 15, 11],
       [ 8, 13,  3, ..., 18, 17,  5],
       [ 5,  2, 13, ..., 14,  1, 14],
       ...,
       [16, 11,  5, ..., 10,  6,  6],
       [10,  8, 12, ..., 12, 13,  2],
       [ 3, 13, 10, ...,  8,  8, 17]])

In [5]:
mse = MSE()

In [6]:
%time mse.forward(target=testdata, pred_val=testdata)

Wall time: 2.04 s


In [79]:
class MSE_parallel():
    """MSE Loss Function
    """

    def __init__(self):
        pass

    @njit(parallel=True)
    def forward(self, target, pred_val):
        """Forward propagation
        https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html

        Args:
            target (samples, n_features): Target value
            pred_val (samples, n_features): Predicted value
        """
        self.target = target
        self.pred_val = pred_val
        diff = self.target - self.pred_val
        self.output = np.sum(diff**2) / (self.target.size * 2)

    def backward(self):
        """Backward propagation
        https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html

        Returns:
            samples, n_features: Gradient of loss
        """
        diff = self.target - self.pred_val
        self.delta_grad = -1 * diff / self.target.size
        return self.delta_grad
    
# @njit(parallel=True)
# def forward(target, pred_val):
#     """Forward propagation
#     https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html

#     Args:
#         target (samples, n_features): Target value
#         pred_val (samples, n_features): Predicted value
#     """
#     target = target
#     pred_val = pred_val
#     diff = target - pred_val
#     output = np.sum(diff**2) / (target.size * 2)


In [80]:
mse_parallel = MSE_parallel()

In [81]:
%time mse_parallel.forward(target=testdata, pred_val=testdata)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at <ipython-input-79-609933cbd498> (18)[0m
[1m
File "<ipython-input-79-609933cbd498>", line 18:[0m
[1m    def forward(self, target, pred_val):
        <source elided>
        """
[1m        self.target = target
[0m        [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'type'>[0m
